# Metaklasy

Wszystko jest obiektem - tak jak instancje mają swoje klasy/typy które określają ich zachowanie, tak klasy mają swoje klasy/typy, które określają ich zachowanie. Są to właśnie metaklasy. Podstawiową metaklasą jest type (tak, to type, które mówi nam jakiego typu są obiekty); type jest metaklasą dla samego siebie.

Metaklasy to funkcje albo klasy, które powinny zwrócić nową klasę 

In [2]:
print type(2)
print type(int)
print type(type)

<type 'int'>
<type 'type'>
<type 'type'>


In [11]:
# metaklasa przyjmuje 3 argumenty -
# nazwę klasy, listę klas bozowych i namespace - atrybuty i metody
def metaklasa(name, bs, ns):
    print 'metaklasa pozdrawia'
    print 'name: ', name
    print 'bs', bs
    print 'ns', ns
    return type(name, bs, ns)

# metaklasę podpina się w pythonie 2 do atrybutu __metaclass__
class A(object):
    __metaclass__ = metaklasa
    
print 30 * '='
class B(A, object):
    __metaclass__ = metaklasa
    x = 23
    def f(self):
        print self

metaklasa pozdrawia
name:  A
bs (<type 'object'>,)
ns {'__module__': '__main__', '__metaclass__': <function metaklasa at 0x10667ade8>}
metaklasa pozdrawia
name:  B
bs (<class '__main__.A'>, <type 'object'>)
ns {'x': 23, '__module__': '__main__', '__metaclass__': <function metaklasa at 0x10667ade8>, 'f': <function f at 0x1066a6758>}


In [13]:
# generalnie metaklasa wcale nie musi wywoływać type

def metaklasa(*a):
    return 2

class A:
    __metaclass__ = metaklasa
    

print A

2


In [33]:
# w praktyce, metaklalsy to klasy, które dziedziczą po type (oczywiscie nie muszą)

class Meta(type):
    def __new__(cls, name, bs, ns):
        # new to classmethod  - cls jest metaklasą Meta
        # przechwytywanie procesu tworzenia klasy - tego co będzie przypisane do zmiennej
        # o znawie name
        print 'tworzenie obiektu'
        # można wpływac na atrybuty, mro, i nazwę klasy, ale teraz nie zrobimy nic
        return super(Meta, cls).__new__(cls, name, bs, ns)
    
    def __init__(self, name , bs, ns):
        # na tym etapie self to juz utworzona klasa
        # inicjalizacja klasy - nazwa jest już nadana, to za późno, żeby to zmienić,
        # tak samo klasy bazowe i atrybuty
        print "inicjalizacja"
        print name, bs, ns
        name = name.upper()
        bs = ()
        ns = {'x': 24}
        return super(Meta, self).__init__(name, bs, ns)
    
    def __call__(self, *a, **kw):
        # tak jak a() wywoływało metodę __call__ z klasy na rzecz tego obiektu,
        # tak A() wywołuje __call__ metaklasy, na rzecz tej klasy
        print 'Meta.__call__'
        return super(Meta, self).__call__(*a, **kw)
        

class A(object):
    pass
    
    
class Aasdf(A):
    __metaclass__ = Meta
    x = 23
    
print Aasdf
print Aasdf.__name__  # __init__ nie wpływa na nazwę klasy
print Aasdf.mro()  # __init__ nie wpływa na mro
print dir(Aasdf)  # __init__ nie wpływa na listę atrybutów
print Aasdf.x  # __init__ nie wpływa na wartosći atrybutów

a = Aasdf()

tworzenie obiektu
inicjalizacja
Aasdf (<class '__main__.A'>,) {'x': 23, '__module__': '__main__', '__metaclass__': <class '__main__.Meta'>}
<class '__main__.Aasdf'>
Aasdf
[<class '__main__.Aasdf'>, <class '__main__.A'>, <type 'object'>]
['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__metaclass__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'x']
23
Meta.__call__


## klasy abstrakcyjne
moduł abc (Abstract Base Classes) zawiera metaklasę ABCMeta, która jest uzywana do tworzenia 
m.in metod abstrakcyjnych, ale równiez kilku innyc hciekawych rzeczy

In [38]:
# można dodac 'virtualne klasy bazowe' przy pomocy funkcji register

import abc


class KlasaBazowa:
    __metaclass__ = abc.ABCMeta
    
    
KlasaBazowa.register(tuple)  # krotka będzie widziana jako podklasa KlasyBazowej


print tuple.mro()  # KlasyBazowej nie ma w mro


print issubclass(tuple, KlasaBazowa)  # ale i tak pokazuje że krotka jest podklasą
print isinstance((), KlasaBazowa)

print issubclass(list, KlasaBazowa)  # lista nei jest tak postrzegana
print isinstance([], KlasaBazowa)

print 40 * '='

class KlasaBazowa2(KlasaBazowa):
    pass

print issubclass(tuple, KlasaBazowa2)  # funkcja register nie działa dla klas dizedziczącyk po
print isinstance((), KlasaBazowa2)     # wirtualnej klasie bazowej

[<type 'tuple'>, <type 'object'>]
True
True
False
False


In [42]:
# alternatywnie można zaimplementować metodę __subclasshook__ jako classmethod

import abc


class KlasaBazowa:
    __metaclass__ = abc.ABCMeta
    
    @classmethod
    def __subclasshook__(cls, instance):
        if instance in [tuple, int]:  # krotki i inty są uważane za podklasy
            return True
        return False


print tuple.mro()  # KlasyBazowej nie ma w mro


print issubclass(tuple, KlasaBazowa)  # ale i tak pokazuje że krotka jest podklasą
print isinstance((), KlasaBazowa)

print issubclass(list, KlasaBazowa)  # lista nei jest tak postrzegana
print isinstance([], KlasaBazowa)

print 40 * '='

class KlasaBazowa2(KlasaBazowa):
    pass

print issubclass(tuple, KlasaBazowa2)  # działą dla klasy pochodnej - register nei działało
print isinstance((), KlasaBazowa2)


# metodę __subclasshook__ i funkcję register możńa łączyć - wtedy "lista podklas"
# to suma zbiorów klas zarejestrowanych i tych określonych przez metodę __subclasshook__

[<type 'tuple'>, <type 'object'>]
True
True
False
False
True
True


In [47]:
# metody abstrakcyjne muszą być nadpisane w klasach bazowych

import abc

class A:
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def f(self):
        pass
    

# a = A()  # error

class B(A):
    pass

# b = B()  # error


class C(B):
    def f(self):
        pass
    
c = C()  # działa

In [51]:
import abc

class A:
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractproperty
    def f(self):
        pass
    

# a = A()  # error

class B(A):
    pass

# b = B()  # error


class C(B):
    @property
    def f(self):
        pass
    
c = C()  # działa